In [4]:
import numpy as np
import pandas as pd
from scipy import stats 
from tqdm.notebook import tqdm
import time
from statsmodels.graphics.gofplots import qqplot

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

import uuid
import zlib
import random

In [5]:
def get_MDE(mu, std, sample_size, n_groups=2, target_share=0.5, r=1, alpha=0.05, beta=0.2):
    """Возвращает MDE для обычной пользовательской метрики, при заданных параметрах теста. 
                                    ТАРГЕТНЫЕ ГРУППЫ ДОЛЖНЫ БЫТЬ РАВНЫ
    mu: float, среднее выборки на исторических данных
    std: float, стан. отклонение выборки на исторических данных
    sample size: int, размер выборки для теста (включает в себя все группы)
    n_groups: int, количество групп в тесте с учетом всех контрольных и таргетных
    target_share: float, доля одной таргетной группы
    r: float, отношение самой маленькой группы к самой большой группе
    alpha: float, уровень ошибки I рода
    beta: float, уровень ошибки II рода

    return: MDE abs и MDE в %
    """
    
    t_alpha = stats.norm.ppf(1 - ((alpha / 2)), loc=0, scale=1)
    comparisons = n_groups - 1
    t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
    sample_ratio_correction = r+2+1/r
    mde = np.sqrt(sample_ratio_correction)*(t_alpha + t_beta) * std / np.sqrt(sample_size*(1-target_share*(comparisons-1)))
    return mde, mde*100/mu

In [6]:
# Параметры
MEAN = 1.82
STD = 2.36

In [7]:
# Решение для mde из статьи
sample_size = 2000
mde = get_MDE(MEAN, STD, sample_size)
print(f'MDE в абс.значениях равно: {mde[0]:0.3f}, в отн. {mde[1]:0.3f}%')

MDE в абс.значениях равно: 0.296, в отн. 16.246%


In [17]:
# Функция для расчета размера групп на основе формул из статьи
def get_sample_size(mu: float=None,
                    std: float=None, 
                    mde: float=None, 
                    n_groups: int=2, 
                    target_share: float=0.5, 
                    r: float=1, 
                    alpha: float=0.05, 
                    beta: float=0.2
                   ):
    """
    Возвращает sample_size для обычной пользовательской метрики, при заданных параметрах теста.
                                    ТАРГЕТНЫЕ ГРУППЫ ДОЛЖНЫ БЫТЬ РАВНЫ
    mu: среднее выборки на исторических данных
    std: стан. отклонение выборки на исторических данных
    n_groups: количество групп в тесте с учетом всех контрольных и таргетных
    target_share: доля одной таргетной группы
    r: отношение самой маленькой группы к самой большой группе
    alpha: уровень ошибки I рода
    beta: уровень ошибки II рода

    return: 
        sample_size: размер выборки для теста (включает в себя все группы)
    """
    
    t_alpha = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
    sample_ratio_correction = r + 2 + 1 / r
    comparisons = n_groups - 1
    mu_diff_squared = (mu - mu * mde)**2
    sample_size = (
        sample_ratio_correction * (
            (t_alpha + t_beta)**2) * (std**2)
    ) / (
        mu_diff_squared * (1 - target_share * (comparisons - 1))
    )
    print(sample_size)
    return int(np.ceil(sample_size))

In [9]:
# Решение для случая с двумя группами - A и B
# n_groups = 2, target_share = 0.5, r = 1
mde = 1.16246
get_sample_size(MEAN, STD, mde)

2001

In [10]:
# Решение для случая с тремя группами - A, B, C 
# Пусть контрольная группа будет самой большой и в ней будет 50% пользователей, тогда таргетные группы будут по 25%
# n_groups = 3, target_share = 0.25, r = 0.5
mde = 1.16246
get_sample_size(MEAN, STD, mde, n_groups = 3, target_share = 0.25, r = 0.5)

3001

In [11]:
3001 * 0.5, 3001 * 0.25

(1500.5, 750.25)

In [12]:
# Решение для случая с четырьмя группами - A, B, C, D
# Пусть контрольная группа будет самой большой и в ней будет 60% пользователей, тогда таргетные группы будут по 10%
# n_groups = 4, target_share = 0.1, r = 0.1 / 0.6
mde = 1.16246
get_sample_size(MEAN, STD, mde, n_groups = 4, target_share = 0.1, r = 0.1 / 0.6)

5105

In [13]:
5105 * 0.6, 5105 * 0.1

(3063.0, 510.5)

In [14]:
# Решение для случая с четырьмя группами - A, B, C, D
# Пусть все группы будут равными
# n_groups = 4, target_share = 0.25, r = 1
mde = 1.16246
get_sample_size(MEAN, STD, mde, n_groups = 4, target_share = 0.25, r = 1)

4001

In [15]:
4001 * 0.25

1000.25

In [18]:
# Решение для случая из статьи https://habr.com/ru/companies/X5Tech/articles/596279/
# n_groups = 2, target_share = 0.5, r = 1
mu_control = 2500
std = 800
mde = 1 + 100 / mu_control
get_sample_size(mu_control, std, mde)

2009.3132119933664


2010

In [19]:
2009.3132119933664 / 2

1004.6566059966832

In [24]:
def get_sample_size_standart(mu=None, std=None, mde=None, alpha=0.05, beta=0.2, n_groups=None, target_share=None, r=None):
    t_alpha = abs(stats.norm.ppf(1 - alpha / 2, loc=0, scale=1))
    t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
    
    mu_diff_squared = (mu - mu * mde) ** 2
    z_scores_sum_squared = (t_alpha + t_beta) ** 2
    disp_sum = 2 * (std ** 2)
    sample_size = int(
        np.ceil(
            z_scores_sum_squared * disp_sum / mu_diff_squared
        )
    )
    return sample_size

In [25]:
get_sample_size_standart(mu_control, std, mde)

1005